# Detect moving objects in the screen

This document is used to analyze whether there are moving or changing objects in the frame, based on openCV.


# Import camera function libraries

After running the following code block, wait a while and wait for the camera to initialize. After the initialization is successful, a 300x300-sized real-time video screen will appear below the code block.

You can right-click on this screen and click `Create New View for Output`, so that you can place the camera screen in the window again. Even if you browse to other part of the document, you can still watch the camera screen at any time. This method applies to other widgets.

The initialization may fail due to running this code block multiple times. The solution is already included in `jetbot.Camera`, you only need to restart the Kernel, but be careful not to use the circle arrow above the tab, chances are the camera will still fail to initialize.

It is a recommended method to restart the Kernel:
In `File Browser` on the left, right-click on the `*.ipynb` file with a green dot in front (the green origin indicates that the Kernel is running), select `Shut Down Kernel`, and you will find a green dot disappears, then close this tab and double-click the `*.ipynb` file that was closed just now to restart the kernel.

Run the following code again, and the camera should be initialized normally.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Motion detection function

The motion detection function is based on openCV. OpenCV is pre-installed in Jetpack, so you can run the following code block to import the required function library directly. If you are not using jetpack, you may need to manually install openCV or imutils in the terminal, and use `sudo pip3 install opencv-python` and `sudo pip3 install imutils` to install the  libraries respectively. If there is no error prompting that these two libraries are missing, you can ignore these and proceed directly The next code block runs.

In [2]:
import cv2
import imutils
import datetime

# avg is used to save a frame of reference picture (background)
# the new picture is compared with it to determine where in the picture has changed.
avg = None


lastMovtionCaptured = datetime.datetime.now()

# Motion detection function
def motionDetect(imgInput):
    global avg, lastMovtionCaptured
    
    # Get the current timestamp.
    timestamp = datetime.datetime.now()
    
    # Convert the frame to black and white, which can increase the efficiency of analysis.
    gray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur the frame to avoid misjudgment caused by noise.
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # If the reference frame (background) has not been obtained, create a new one.
    if avg is None:
        avg = gray.copy().astype("float")
        return imgInput

    # background update.
    cv2.accumulateWeighted(gray, avg, 0.5)
    
    # Compare the difference between the new frame and the background.
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    # Get the outline of the changed area in the frame.
    thresh = cv2.threshold(frameDelta, 5, 255,
        cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # There may be more than one area changes in the frame, so you need to use a for loop to get all the contours.
    for c in cnts:
        # The default here is 30, which is the threshold of the change area. We only analyze the area greater than 800.
        # The smaller the value, the more sensitive the motion detection, but it may also detect meaningless noise.
        if cv2.contourArea(c) < 30:
            continue

        # Draw elements, including rectangle and text.
        (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
        cv2.rectangle(imgInput, (mov_x, mov_y), (mov_x+mov_w, mov_y+mov_h), (128, 255, 0), 1)

        # Save the current timestamp to mark the time when the change is detected.
        lastMovtionCaptured = timestamp

    # In order to avoid the high flickering frequency of drawing elements
    # within 0.5 seconds after the motion ends, elements stay.
    if (timestamp - lastMovtionCaptured).seconds >= 0.5:
        cv2.putText(imgInput,"Motion Detecting",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(128,255,0),1,cv2.LINE_AA)
    else:
        cv2.putText(imgInput,"Motion Detected",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,128,255),1,cv2.LINE_AA)
    
    # Return to the processed frame.
    return imgInput

# Process video frames and display

After running the following code, you can see that the color of the frame has changed, indicating that the video screen has been successfully processed by the `motionDetect()` function.

In [3]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(motionDetect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

At this point you have run all the code. When an object moves or changes in the frame, the text content will change, and a green rectangle will mark the changed area.

# Turn off this processing and stop the camera
Run the following code to turn off the image processing function.

In [4]:
camera.unobserve(execute, names='value')

Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [4]:
camera.stop()